# CMDP＋モデルフリー＋強リグレット

参考：
* [Towards Achieving Sub-linear Regret and Hard Constraint Violation in Model-free RL](https://proceedings.mlr.press/v238/ghosh24a.html)

表記：　
$$
\begin{aligned}
\operatorname{Regret}(K)&=\sum_{k=1}^K\left(V_{r, 1}^*\left(x_1\right)-V_{r, 1}^{\pi_k}\left(x_1\right)\right)\\
\operatorname { Violation }_H(K)&:=\sum_{k=1}^K\left(b-V_{g, 1}^{\pi_k}\left(x_1\right)\right)_{+}
\end{aligned}
$$
* $r$は報酬．$g$は効用関数
* エピソディックなCMDPを考えます．表記は後はノリでわかるはず．
* Linear MDPを考えます：
    * $\mathbb{P}_h\left(x^{\prime} \mid x, a\right)=\left\langle\phi(x, a), \mu_h\left(x^{\prime}\right)\right\rangle$
    * $r_h(x, a)=\left\langle\phi(x, a), \theta_{r, h}\right\rangle \quad g_h(x, a)=\left\langle\phi(x, a), \theta_{g, h}\right\rangle$

## アルゴリズム
* ボーナスの係数：$C_1 d H \sqrt{\log (4 \log |\mathcal{A}| d KH / p)}$
* $Y_k$の増やし方：$\eta =1 /\left(d^{H-1} K^{1.5 H} \log (|\mathcal{A}|)^H\right)$
* 学習率：$\alpha=(\log (|\mathcal{A}|) \sqrt{K}) /(4 H)$

for episode $k=1\dots K$

* $Y_k = 0$とします．
* while $Y_k \leq \sqrt{K}$で次を実行します．While-Loopを走らせて，最も良い$Y$の値を見つけます．：
  * 各ステップ$h = H, \dots, 1$について，
    * （状態行動のカウント）：$\Lambda_h^k=\sum_{\tau=1}^{k-1} \phi\left(x_h^\tau, a_h^\tau\right) \phi\left(x_h^\tau, a_h^\tau\right)^\top+\lambda \mathbf{I}$．**こいつは$Y$の値に関係ないので，While-loop中では不変**
    * ===== **ここから下は$Y$の値で変化するので注意** =====
    * （Qr関数の回帰）：$w_{r, h}^k=\left(\Lambda_h^k\right)^{-1}\left[\sum_{\tau=1}^{k-1} \phi\left(x_h^\tau, a_h^\tau\right)\left[r_h\left(x_h^\tau, a_h^\tau\right)+ V_{r, h+1}^k\left(x_{h+1}^\tau\right) \right]\right]$
    * （Qg関数の回帰）$w_{g, h}^k=\left(\Lambda_h^k\right)^{-1}\left[\sum_{\tau=1}^{k-1} \phi\left(x_h^\tau, a_h^\tau\right)\left[g_h\left(x_h^\tau, a_h^\tau\right)+ V_{g, h+1}^k\left(x_{h+1}^\tau\right) \right]\right]$
    * $Q_{r, h}^k(\cdot, \cdot) \quad=\quad \min \left\{\left\langle w_{r, h}^k, \phi(\cdot, \cdot)\right\rangle + \beta\left(\phi(\cdot, \cdot)^\top\left(\Lambda_h^k\right)^{-1} \phi(\cdot, \cdot)\right)^{1 / 2}, H\right\}$
    * $Q_{g, h}^k(\cdot, \cdot) \quad=\quad \min \left\{\left\langle w_{g, h}^k, \phi(\cdot, \cdot)\right\rangle + \beta\left(\phi(\cdot, \cdot)^\top\left(\Lambda_h^k\right)^{-1} \phi(\cdot, \cdot)\right)^{1 / 2}, H\right\}$
    * 方策関数の更新：$\pi_{h, k}(\cdot \mid x)\propto \exp\left(\alpha\left(Q_{r, h}^k+Y_k Q_{g, h}^k\right)\right)$
    * $V_{r, h}^k(\cdot)=\sum_a \pi_{h, k}(a \mid \cdot) Q_{r, h}^k(\cdot, a)$
    * $V_{g, h}^k(\cdot)=\sum_a \pi_{h, k}(a \mid \cdot) Q_{g, h}^k(\cdot, a)$
  * もし$V_{g, 1}^k(x_1) \geq b$ならば，while文をbreak
  * そうでなければ$Y_k \leftarrow Y_k + \eta$
* もし$Y_k > \sqrt{K}$ならば，$Y_k = \sqrt{K}$
* for step $h = 1, \dots, H$で，サンプルを収集


---

**コメント**
* [モデルフリーのQ学習](RL_UCB_H_regret_proof.ipynb)と違ってQ関数を学習率で更新してない？Softmaxが効いてるのか？
* 後ろから更新するので，実質これは方策評価 → 方策更新してるはず．
* While-loopによって最も良いペナルティ係数 $Y_k$ を探している．

このとき，$Y_k$の値には3パターンあります：
1. $Y_k=0, V_{g, 1}^k\left(x_1\right) \geq b$のとき：報酬の最大化だけに集中すれば良い
2. $\sqrt{K} \geq Y_k>0$, and $V_{q, 1}^k\left(x_1\right) \geq b$のとき：このとき，$\eta$と$\alpha$をいい感じに調整すると，$V_{q, 1}^k\left(x_1\right) \leq b+\mathcal{O}\left(K^{-1}\right)$が成り立つらしい．つまり，そんなに制約に力を入れすぎないように調整してるっぽい．
3. $Y_k=\sqrt{K}$, and $V_{g, 1}^k\left(x_1\right)<b$のとき：このときはdual variableのUpper boundに到達してます．このUpper boundのとき，$\sqrt{K}$リグレットかつ$\sqrt{K}$違反になるらしい．

## リグレット解析

表記：
* $Q_{r, h}^{k, Y}, Q_{g, h}^{k, Y}, V_{r, h}^{k, Y}, V_{g, h}^{k, Y}, w_{r, h}^{k, Y}, w_{g, h}^{k, Y}$
  * アルゴリズム中で得られた$Q$値とか．アルゴリズム中では$Y_k$を使っているが，多分ここは$Y$を固定して表記してると思われる．


まず次のリグレットに分解します：

$$
\begin{aligned}
\operatorname{Regret}(K)= & \underbrace{\sum_{k=1}^K\left(V_{r, 1}^*\left(x_1\right)-V_{r, 1}^k\left(x_1\right)\right)}_{\mathcal{T}_1}+ \\
& \underbrace{\sum_{k=1}^K\left(V_{r, 1}^k\left(x_1\right)-V_{r, 1}^{\pi_k}\left(x_1\right)\right)}_{\mathcal{T}_2}
\end{aligned}
$$

$\mathcal{T}_2$はAzuma-Hoeffdingを使えば簡単にバウンドできます．
$\mathcal{T}_1$は次でバウンドされます：

$$
\begin{aligned}
\mathcal{T}_1 \leq & \underbrace{\sum_{k=1}^K\left(V_{r, 1}^*\left(x_1\right)+Y_k V_{g, 1}^*\left(x_1\right)-V_{r, 1}^k\left(x_1\right)-Y_k V_{g, 1}^k\left(x_1\right)\right)}_{\mathcal{T}_3} \\
& +\underbrace{\sum_{k=1}^K Y_k\left(V_{g, 1}^k\left(x_1\right)-b\right)}_{\mathcal{T}_4}
\end{aligned}
$$

これは$Y_k V^k_{g, 1}(x_1)$を±して，さらに$V^*_{g, 1}$が$b$よりでかいことを使ってます．
* $\mathcal{T}_3$でOptimism的なことが必要になります．今回はSoftmaxを使ってるのでさらにめんどくさいです．ここがキモ
* $\mathcal{T}_4$のバウンドも結構ややこしいです．$b$を超えてしまう回数をバウンドすることでなんとかします．


## 便利な定理など

---

任意の方策$\pi$について，$Q_{j, h}^\pi=\left\langle\phi(x, a), w_{j, h}^\pi\right\rangle$を満たす重みを$w_{j, h}$とします．
このとき，

$$
\left\|w_{j, h}^\pi\right\| \leq 2 H \sqrt{d}
$$

が成立します．

**証明**

$$
\begin{aligned}
Q_{j, h}^\pi(x, a) & =j_h(x, a)+\mathbb{P}_h V_{j, h}^\pi(x, a) \\
& =\left\langle\phi(x, a), \theta_{j, h}\right\rangle+\int_{\mathcal{S}} V_{j, h+1}^\pi\left(x^{\prime}\right)\left\langle\phi(x, a), d \mu_h\left(x^{\prime}\right)\right\rangle \\
& =\left\langle\phi(x, a), w_{j, h}^\pi\right\rangle
\end{aligned}
$$

ここで，$w_{j, h}^\pi=\theta_{j, h}+\int_{\mathcal{S}} V_{j, h+1}^\pi\left(x^{\prime}\right) d \mu_h\left(x^{\prime}\right)$です．$\left\|\theta_{j, h}\right\| \leq \sqrt{d}$, and $\left\|\int_{\mathcal{S}} V_{j, h+1}^\pi\left(x^{\prime}\right) d \mu_h\left(x^{\prime}\right)\right\| \leq H \sqrt{d}$なので，証明終わりです．

---

任意の$(k, h, Y)$について，

$$
\left\|w_{j, h}^{k, Y}\right\| \leq 2 H \sqrt{d k / \lambda}
$$

が成立します．

**証明** 省略．

---

**Softmaxと貪欲方策の差**

$$
\bar{V}_h^{k, Y}(\cdot)=\max _a\left[Q_{r, h}^{k, Y}(\cdot, a)+Y Q_{g, h}^{k, Y}(\cdot, a)\right]
$$
に対して，
$$
\bar{V}_h^{k, Y}(x)-V_h^{k, Y}(x) \leq \frac{\log |\mathcal{A}|}{\alpha}
$$
が成立します．つまり，Softmaxの係数$\alpha$をめっちゃでかくして貪欲方策に近づけると，貪欲方策との差も小さくなります．

---


## $\mathcal{T}_3$のバウンド

**バウンドの手順**

$h=H$から初めて，帰納法で示します．

1. まずは$\left|\left\langle\phi(x, a), w_{j, H}^{k, Y}(x, a)\right\rangle-Q_{j, H}^\pi(x, a)\right| \leq \beta \sqrt{\phi(x, a)^T\left(\Lambda_H^k\right)^{-1} \phi(x, a)}$を示します．$Q^{k, Y}_{j, H+1}=0=Q^{\pi}_{j, H+1}$を使います．

---

任意のエピソード$k$かつ$0\leq Y\leq \sqrt{K}$について，次が確率$1-p$以上で成立します：

$$
\left(V_{r, 1}^*\left(x_1\right)-V_{r, 1}^{k, Y}\left(x_1\right)+Y V_{g, 1}^*\left(x_1\right)-
Y V_{g, 1}^{k, Y}\left(x_1\right)\right) \leq \frac{\log (|\mathcal{A}|) H}{\alpha}
$$

**証明**

まず，次のイベント$\mathcal{E}$が$1-p$以上の確率で成立するような定数$C_2$が存在します．（Potential lemmaで使う？）TODO: これの証明．結構ダルそう．

$$
\left\|\sum_{\tau=1}^{k-1} \phi_{j, h}^\tau\left[V_{j, h+1}^{k, Y}\left(x_{h+1}^\tau\right)-\mathbb{P}_h V_{j, h+1}^{k, Y}\left(x_h^\tau, a_h^\tau\right)\right]\right\|_{\left(\Lambda_h^k\right)^{-1}} \leq C_2 d H \sqrt{\chi}
$$

ここで，$\chi=\log \left[4\left(C_1+1\right) \log (|\mathcal{A}|) d T / p\right]$です．


また，次が成立します．（上と合体させてPotential lemmaで使う？）TODO: これの証明．ダルそう
$$
\left\langle\phi(x, a), w_{j, h}^{k, Y}\right\rangle-Q_{j, h}^\pi(x, a)=\mathbb{P}_h\left(V_{j, h+1}^{k, Y}-V_{j, h+1}^\pi\right)(x, a)+\Delta_h^k(x, a)
$$
ここで，$\left|\Delta_h^k(x, a)\right| \leq \beta \sqrt{\phi(x, a)^T\left(\Lambda_h^k\right)^{-1} \phi(x, a)}$です．


---
